In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import read_csv, concat
sns.set_theme(style='white', context='notebook', font_scale=1.33)

## Section 1: Item Selection

In [2]:
## Prepare study 1.
df1 = read_csv('data/stats.csv')
df1 = df1[df1.Group.isin(['Ad','OA','MA'])].groupby('Item').agg({'N':'sum','Mean.correct':'mean'})
df1 = df1.rename(columns={'N':'n1','Mean.correct':'a1'})
df1['a1'] *= 1e-2

## Prepare study 2.
df2 = read_csv('data/study02/data.csv')
df2 = df2.groupby('item').agg({'subject':'count','accuracy':'mean'})
df2 = df2.rename(columns={'subject':'n2','accuracy':'a2'})

## Prepare study 3.
df3 = read_csv('data/study03/data.csv')
df3 = df3.groupby('item').agg({'subject':'count','accuracy':'mean'})
df3 = df3.rename(columns={'subject':'n3','accuracy':'a3'})

## Prepare study 4.
df4 = read_csv('data/study04/data.csv')
df4 = df4.groupby('item').agg({'subject':'count','accuracy':'mean'})
df4 = df4.rename(columns={'subject':'n4','accuracy':'a4'})

## Merge DataFrames.
data = df1.merge(df2, left_index=True, right_index=True, how='outer')
data = data.merge(df3, left_index=True, right_index=True, how='outer')
data = data.merge(df4, left_index=True, right_index=True, how='outer')

## Compute total sample.
f = lambda x: np.average(x[['a1','a2','a3','a4']].dropna(), weights=x[['n1','n2','n3','n4']].dropna())
data['Total'] = data.filter(regex='n').sum(axis=1)
data['Mean'] = data.apply(f, 1)

## Add dimension.
dimension = read_csv('data/dimensionality.csv', index_col=0)
dimension = dimension.rename(columns={'Dimensionality Score':'dimension'})
data = data.merge(dimension, left_index=True, right_index=True, how='outer')

## Section 2: Item Order

In [3]:
## Restrict to items of interest.
exclude = [1, 2, 3, 4, 5, 7, 8, 9, 32, 33, 38, 41, 43, 48, 57, 68]
data = data.loc[np.setdiff1d(np.arange(1,81), exclude)]

## Sort items.
data = data.sort_values(['dimension','Total'], ascending=[True, False])

print(data.index.values.reshape([-1,4]))

[[ 6 19 20 22]
 [25 28 39 47]
 [49 50 51 61]
 [65 70 71 10]
 [15 11 18 13]
 [27 31 34 37]
 [56 58 62 69]
 [77 16 17 12]
 [23 40 42 53]
 [60 72 73 79]
 [80 14 26 52]
 [59 63 67 74]
 [76 36 44 46]
 [54 55 64 75]
 [21 24 29 30]
 [45 78 35 66]]
